In [70]:
import fasttext
import fasttext.util
import torch
import math
from model import gan
from sklearn.utils import shuffle
import numpy as np

In [12]:
### VARIABLES & ADMINISTRATIVE STUFF ###
# System
#dataset_path = '/media/daniel/Elements/FastText_Data/'  # In case dataset is stored somewhere else, e.g. on hard-drive
dataset_path = '/media/daniel/Elements/FastText_Data/'  #data in same directory
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Network
embedding_dim = 300
internal_dim = 300
hidden = 300

# Train hyperparameters
epochs = 2
batch_size = 32
vocab_size = 2000
num_minibatches = vocab_size // batch_size
real_label, fake_label = 1, 0
languages = {'src': ['de', 'nl'], 'trgt': 'en'}  # Target language to be indicated in last position

In [13]:
def cleaned_vocab(vocab, vocab_size):
    # Remove all punctuation tokens while valid nr of tokens is insufficient yet for having full vocab size
    # TODO & possibly reserve testing vocab

    # Return clean & restricted vocab
    words = vocab.words[:vocab_size]              # Y (labels)
    vects = [vocab[word] for word in words]       # X (input data)

    return vects, words

In [14]:
def add_lang_to_vocab(lang_type, lang_id, vocab_size, vocabs):
    # Get dataset
    if dataset_path == './':
        fasttext.util.download_model(lang_id)  # Download word embedding vector data if not available
    vocab = fasttext.load_model(dataset_path + 'cc.' + lang_id + '.300.bin')  # Load language data

    # Add train data (embedding-vectors) and labels (words) to vocab
    x, y = cleaned_vocab(vocab, vocab_size)
    vocabs[lang_type][lang_id] = {'x': torch.tensor(x), 'y': y}

    return vocabs

In [15]:
def load_vocab(languages):
    nr_src_langs = len(languages)
    vocabs = {'src': {}, 'trgt': {}}
    
    for language in languages['src']:
        vocabs = add_lang_to_vocab('src', language, vocab_size, vocabs)
        
    language = languages['trgt']
    vocabs = add_lang_to_vocab('trgt', language, vocab_size, vocabs)

    print('Successfully loaded language models.')
    return vocabs

In [16]:
    # load vocab
    vocabs = load_vocab(languages)

Successfully loaded language models.


In [91]:
def main():
    
    NLLLoss = torch.nn.NLLLoss()
    nr_src_langs = len(vocabs['src'])
    print('Nr source languages:', nr_src_langs)
    print('Nr target languages:', len(vocabs['trgt']))
    
    # Get bilingual dictionary for evaluating train loss or at least testing
    dicts = dict()
    #TODO

    # Set up model architecture
    net = gan.GAN(embedding_dim, internal_dim, hidden, languages['src'])

    # Get optimizers; 1 per source language and 1 for target language
    optims_g = {}
    for language in languages['src']:

        #params = net.generator.encoders[language].parameters() + net.generator.decoder.parameters()
        optims_g[language] = torch.optim.Adam([{'params': net.generator.encoders[language].parameters()},{'params': net.generator.decoder.parameters()}],lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

    optim_d = torch.optim.Adam(net.discriminator.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

    # Train
    train_loss_real_d, train_loss_fake_d, train_loss_g = [], [], []
    eval_loss = [], []
    for epoch in range(epochs):
        print('Epoch ', epoch, '/', epochs)
        loss_real_total_d, loss_fake_total_d, loss_total_g = 0., 0., 0.

        # Shuffle data #
        # Source languages
        for lang in languages['src']:
            vocabs['src'][lang]['x'], vocabs['src'][lang]['y'] = shuffle(np.array(vocabs['src'][lang]['x']), np.array(vocabs['src'][lang]['y']))
            vocabs['src'][lang]['x'] = torch.from_numpy(vocabs['src'][lang]['x'])
            #vocabs['src'][lang]['y'] = torch.from_numpy(vocabs['src'][lang]['y']) # We don't really need that except for eval
        # Target language
        lang = languages['trgt'] # Retrieve language id
        vocabs['trgt'][lang]['x'], vocabs['trgt'][lang]['y'] = shuffle(np.array(vocabs['trgt'][lang]['x']), np.array(vocabs['trgt'][lang]['y']))
        vocabs['trgt'][lang]['x'] = torch.from_numpy(vocabs['trgt'][lang]['x'])

        # Train #
        for batch in range(num_minibatches):
            print('Epoch ', epoch, ', Batch ', batch, '/', num_minibatches)

            # Update discriminator #
            # All-real minibatch
            net.discriminator.zero_grad()
            x = vocabs['trgt'][languages['trgt']]['x'][batch * batch_size:(batch + 1) * batch_size]#.to(device)
            y_true = torch.full((batch_size,), real_label).long() #device=device  # TODO: could probs be converted to long straight away when reading in, already...
            y_pred = net.discriminator(x)
            # Loss proportional to discriminator's probability of correctly distinguishing TP and FP
            loss_real = NLLLoss(torch.log(y_pred), y_true)  # NLLLoss needs log(prob_distribution)
            loss_real.backward()
            loss_real_total_d += loss_real

            # One minibatch per source language
            translations = {}
            loss_fake_batch_avg = 0.
            for language in languages['src']:
                # All-real minibatch
                net.discriminator.zero_grad()
                x = vocabs['src'][language]['x'][batch * batch_size:(batch + 1) * batch_size]#.to(device)
                x = net.generator(x, language)
                translations[language] = x
                y_true = torch.full((batch_size,), fake_label).long() #, device=device
                y_pred = net.discriminator(x.detach())      # Detach to avoid computing grads for generator
                # Loss proportional to discriminator's probability of correctly distinguishing TP and FP
                loss_fake = NLLLoss(torch.log(y_pred), y_true)  # NLLLoss needs log(prob_distribution)
                loss_fake_batch_avg += loss_fake
                loss_fake.backward()    # Compute gradients only for discriminator
            optim_d.step()              # Weight update
            loss_fake_total_d += (loss_fake_batch_avg/nr_src_langs)

            # Update generator #
            # Compute gradients
            loss_fake_batch_avg = 0.
            for language in languages['src']:
                net.generator.encoders[language].zero_grad()
                x = translations[language]
                y_true = torch.full((batch_size,), real_label).long() #device=device
                y_pred = net.discriminator(x)
                #loss_fake = net.loss(y_pred, y_true, 'gen')
                # Loss proportional to discriminator's probability of confusing TP and FP
                loss_fake = NLLLoss(torch.log(y_pred), y_true)
                loss_fake_batch_avg += loss_fake
                loss_fake.backward()
            loss_total_g += (loss_fake_batch_avg / nr_src_langs)

            # TODO: possibly average decoder's gradients

            # Perform weight updates
            for language in languages['src']:
                optims_g[language].step()

        # Document accumulated losses per epoch
        train_loss_real_d.append(loss_real_total_d)
        train_loss_fake_d.append(loss_fake_total_d)
        train_loss_g.append(loss_total_g)
        
        print('Progress: ', loss_real_total_d, loss_fake_total_d, loss_total_g)

        # TODO: Similarity metric-based evaluation per epoch?

    # TODO: Final evaluation

    # TODO: Store model

if __name__ == "__main__":
    # execute only if run as a script
    main()



Nr source languages: 2
Nr target languages: 1
Epoch  0 / 10
Epoch  0 , Batch  0 / 62
Epoch  0 , Batch  1 / 62
Epoch  0 , Batch  2 / 62
Epoch  0 , Batch  3 / 62
Epoch  0 , Batch  4 / 62
Epoch  0 , Batch  5 / 62
Epoch  0 , Batch  6 / 62
Epoch  0 , Batch  7 / 62
Epoch  0 , Batch  8 / 62
Epoch  0 , Batch  9 / 62
Epoch  0 , Batch  10 / 62
Epoch  0 , Batch  11 / 62
Epoch  0 , Batch  12 / 62
Epoch  0 , Batch  13 / 62
Epoch  0 , Batch  14 / 62
Epoch  0 , Batch  15 / 62
Epoch  0 , Batch  16 / 62
Epoch  0 , Batch  17 / 62
Epoch  0 , Batch  18 / 62
Epoch  0 , Batch  19 / 62
Epoch  0 , Batch  20 / 62
Epoch  0 , Batch  21 / 62
Epoch  0 , Batch  22 / 62
Epoch  0 , Batch  23 / 62
Epoch  0 , Batch  24 / 62
Epoch  0 , Batch  25 / 62
Epoch  0 , Batch  26 / 62
Epoch  0 , Batch  27 / 62
Epoch  0 , Batch  28 / 62
Epoch  0 , Batch  29 / 62
Epoch  0 , Batch  30 / 62
Epoch  0 , Batch  31 / 62
Epoch  0 , Batch  32 / 62
Epoch  0 , Batch  33 / 62
Epoch  0 , Batch  34 / 62
Epoch  0 , Batch  35 / 62
Epoch  0 , Bat

Epoch  4 , Batch  56 / 62
Epoch  4 , Batch  57 / 62
Epoch  4 , Batch  58 / 62
Epoch  4 , Batch  59 / 62
Epoch  4 , Batch  60 / 62
Epoch  4 , Batch  61 / 62
Progress:  tensor(nan, grad_fn=<AddBackward0>) tensor(nan, grad_fn=<AddBackward0>) tensor(nan, grad_fn=<AddBackward0>)
Epoch  5 / 10
Epoch  5 , Batch  0 / 62
Epoch  5 , Batch  1 / 62
Epoch  5 , Batch  2 / 62
Epoch  5 , Batch  3 / 62
Epoch  5 , Batch  4 / 62
Epoch  5 , Batch  5 / 62
Epoch  5 , Batch  6 / 62
Epoch  5 , Batch  7 / 62
Epoch  5 , Batch  8 / 62
Epoch  5 , Batch  9 / 62
Epoch  5 , Batch  10 / 62
Epoch  5 , Batch  11 / 62
Epoch  5 , Batch  12 / 62
Epoch  5 , Batch  13 / 62
Epoch  5 , Batch  14 / 62
Epoch  5 , Batch  15 / 62
Epoch  5 , Batch  16 / 62
Epoch  5 , Batch  17 / 62
Epoch  5 , Batch  18 / 62
Epoch  5 , Batch  19 / 62
Epoch  5 , Batch  20 / 62
Epoch  5 , Batch  21 / 62
Epoch  5 , Batch  22 / 62
Epoch  5 , Batch  23 / 62
Epoch  5 , Batch  24 / 62
Epoch  5 , Batch  25 / 62
Epoch  5 , Batch  26 / 62
Epoch  5 , Batch  2

Epoch  9 , Batch  41 / 62
Epoch  9 , Batch  42 / 62
Epoch  9 , Batch  43 / 62
Epoch  9 , Batch  44 / 62
Epoch  9 , Batch  45 / 62
Epoch  9 , Batch  46 / 62
Epoch  9 , Batch  47 / 62
Epoch  9 , Batch  48 / 62
Epoch  9 , Batch  49 / 62
Epoch  9 , Batch  50 / 62
Epoch  9 , Batch  51 / 62
Epoch  9 , Batch  52 / 62
Epoch  9 , Batch  53 / 62
Epoch  9 , Batch  54 / 62
Epoch  9 , Batch  55 / 62
Epoch  9 , Batch  56 / 62
Epoch  9 , Batch  57 / 62
Epoch  9 , Batch  58 / 62
Epoch  9 , Batch  59 / 62
Epoch  9 , Batch  60 / 62
Epoch  9 , Batch  61 / 62
Progress:  tensor(nan, grad_fn=<AddBackward0>) tensor(nan, grad_fn=<AddBackward0>) tensor(nan, grad_fn=<AddBackward0>)
